In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# The chess interface and game loop

### Dependencies

In [ ]:
%pip install python-chess

import chess
import chess.svg # TODO remove?
import chess.polyglot # TODO remove?
import chess.gaviota # TODO remove?
import time
import random
from IPython.display import clear_output, SVG, display, HTML
from typing import Callable, Union

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


#### chess.Board.push_legal
Pushes a move to a board's move stack only if it is a valid, legal move. If the move is illegal or invalid, a ValueError is raised and the board state remains unaffected.

###### <b><u>Arguments</u></b>
<b>move</b> <i>(chess.Move)</i> : The move that should be pushed to the move stack.

###### <b><u>Side effects</u></b>
Pushes a move to the board's move stack, assuming the move was valid and legal.

In [ ]:
def push_legal(self, move: chess.Move):
    if move not in self.legal_moves:
        raise ValueError('Illegal move')
    self.push(move)

chess.Board.push_legal = push_legal
del push_legal

---
<br>

## Game class
Lets the user play a game of chess against the AI using a virtual chess board UI. The game may use any algorithm for the AI to make its move.
<br><br>

### Class variables
    HTML_CONTENT : String
        The content which is displayed next to the chess board. 
    HTML_HEADER: String
        The Header of the HTML-content.
    BOARD_DISPLAY_SIZE : Integer
        The size of the board in pixels.

    ---

    board : chess.Board
        The chess board instance that's used for the game.
    make_move : dictionary(chess.COLOR : Callable)
        A dictionary of algorithms the AI uses to make a move according to the      
        color-representation of the AI.
    is_ai : dictionary(chess.COLOR : Boolean)
        A dictionary, where the chess.COLOR indicates, whether or not this color is controlled by
        an AI.
    move_times : dictionary(chess.COLOR : List)
        A dictionary with Lists, inheriting the times an AI takes to move according to the 
        chess.COLOR the AI possesses.
    opening_book : String
        A String, representing a path to an opening library.
    is_first_move : Boolean
        A boolean, representing whether or not the first move has already been executed
    
    
<br>

### \_\_init\_\_
    The constructor for the Game class. Gets called whenever a new Game instance is created.
<br>

##### Arguments
    make_move_algorithm_white : Callable(chess.Board) [Optional] (Default behavior : None)
        The function that the AI controlling the white pieces should use to make a new move.
    make_move_algorithm_black : Callable(chess.Board) [Optional] (Default behavior: None)
        The function that the AI controlling the black pieces should use to make a new move.
    opening_book: String [Optional] (Default behavior : None)
        A String representing the path to the desired opening library the AIs will use.

##### Returns
    N/A

##### Side effects
    If no AI is selected as at least one player, the game will exit. At least one AI is required   
    to play the game.
<br><br>

### play
    Plays the game.
    Depending on the input arguments there are different game-states and features:
    1. (AI, AI):
        - Displays the chess board visually.
        - Lets the AI make a move based on the board state and specified algorithm.
        - Lets the other AI make a move based on the board state and specified algorithm.
    2. (AI, Player) - (Player, AI):
        - Displays the chess board visually.
        - Lets the AI make a move based on the board state.
        - Lets the human player input their move and pushes it to the board 
        (if it's valid and legal).
    3. (Player, Player):
        - Game will exit -> At least one AI is required to play the game

##### Arguments
    N/A

##### Returns
    N/A

##### Side effects
    - The state of the board in the Game class is changed.
    - The state of is_first_move is changed after the first move.
    - The times the AIs take to make a move is tracked and saved.
    - The chess board is visually displayed.
<br><br>

### display
    Displays the chess board graphically. Furthermore, custom HTML is displayed next to the board.

##### Arguments
    N/A

##### Returns
    N/A

##### Side effects
    - The current visual output is overwritten.
<br> <br>

### get_check_squares
    checks for piecs or squares giving check.
#### Arguments
    N/A

#### Returns
    chess.Square | None (if no square sets gives check)

##### Side effects
    - If get_check_squares returns a value, the display function will indicate the square giving
      check with a red background
    
<br><br>

### make_opening_move
    Makes a random move from the opening book, if available. If not, no move is made.
#### Arguments
    N/A
#### Returns
    True: A move from the opening book was made.
    False: A move from the opening book was not made.
#### Side effects
    N/A

### get_ai_state
    Define the AIs for the game. Dependent on the ai_algorithms given as arguments in game.play()
    this functions defines the player state for the different colors. (AI or Player). If the 
    "player" is defined as an ai, the dictionary value to the coresponding chess.Color is defined 
    as True.
#### Arguments
    N/A
#### Returns
    {chess.Color: Bool} - Dict
#### Side effects
    N/A

### make_move_human
    If the "player" is human, this method enables the human user to make a move through the 
    input-console.
#### Arguments
    N/A
#### Return
    N/A
#### Side effects
    A move is pushed onto the move_stack and it is the next players turn.

In [ ]:
class Game:
    # Constants
    HTML_CONTENT = 'html stuff'
    HTML_HEADER = 'Header'
    BOARD_DISPLAY_SIZE = 500

    # Board variables
    board = None

    # AI and player declarations
    make_move = None
    is_ai = None

    # Testing declarations
    move_times = None

    # Opening book and endgame tablebase declarations
    opening_book = None
    endgame_tablebase_dir = None
    endgame_tablebase = None

    def __init__(
            self,
            make_move_algorithm_white: Callable=None,
            make_move_algorithm_black: Callable=None,
            opening_book: str='Resources/baron30.bin',
            endgame_tablebase_dir: str='/datasets/gaviota'
    ):
        self.opening_book = opening_book
        self.endgame_tablebase_dir = endgame_tablebase_dir

        self.board = chess.Board()
        self.board.ending = False

        self.move_times = { chess.WHITE: [], chess.BLACK: [] }
        self.make_move = { chess.WHITE: make_move_algorithm_white, chess.BLACK: make_move_algorithm_black }
        self.is_ai = self.get_ai_state()

        if self.endgame_tablebase_dir is not None:
            self.endgame_tablebase = chess.gaviota.open_tablebase(self.endgame_tablebase_dir)

        # The game requires that at least one player is an AI - it is not a multiplayer chess game
        if not self.is_ai[chess.WHITE] and not self.is_ai[chess.BLACK]:
            raise ValueError('At least one player must be an AI')


In [ ]:
def play(self) -> None:
    while not self.board.is_game_over():
        self.display()
        if self.is_ai[self.board.turn]:
            # Start time
            time_stamp = time.time()
            if not self.make_opening_move():
                self.make_move[self.board.turn](self.board, self.board.turn, self.endgame_tablebase)
            # Find time difference
            time_stamp = time.time() - time_stamp
            # Add time to move_times
            self.move_times[self.board.turn].append(time_stamp)
        else:
            self.make_move_human()

        if self.endgame_tablebase is not None:
            try:
                self.HTML_CONTENT = ''
                self.HTML_CONTENT += 'DTM:\n'
                self.HTML_CONTENT += str(self.endgame_tablebase.probe_dtm(self.board)) + '\n'
                self.HTML_CONTENT += 'WDL:\n'
                self.HTML_CONTENT += str(self.endgame_tablebase.probe_wdl(self.board)) + '\n'
            except KeyError:
                pass
        else:
            self.HTML_CONTENT = 'wut'

        self.board.check_and_set_ending()
        self.display()

    winner_message = 'It\'s a draw!' if self.board.outcome() is None or self.board.outcome().winner is None \
        else f'{"White" if self.board.outcome().winner == chess.WHITE else "Black"} is the winner!'
    print(f'The game has ended. {winner_message}\n')
    print(self.board.outcome()) 

Game.play = play
del play

In [ ]:
def display(self) -> None:
    clear_output(wait=True)
    try:
        last_move = self.board.peek()
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE, lastmove=last_move, check=self.get_check_square()) 
    except IndexError:
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE)
    table = '<table><th>Board</th><th>{}</th><tr><td><div style=\"vertical-align: top; text-align: left\">{}</div></td><td>{}</td></div></tr></table>'
    display(HTML(table.format(self.HTML_HEADER, board_visual, self.HTML_CONTENT)))

Game.display = display
del display

In [ ]:
def get_check_square(self) -> chess.Square:
    if self.board.is_check():
        return self.board.checkers().pop()
    else:
        return None
            
Game.get_check_square = get_check_square
del get_check_square

In [ ]:
def make_opening_move(self) -> bool:
    if self.opening_book is None:
        return False

    with chess.polyglot.open_reader(self.opening_book) as reader:
        possible_moves = [ entry.move for entry in reader.find_all(self.board) ]
        if len(possible_moves) > 0:
            self.board.push(random.choice(possible_moves))
            return True

    return False

Game.make_opening_move = make_opening_move
del make_opening_move

In [ ]:
def get_ai_state(self) -> dict:
    return {
        color: self.make_move[color] is not None
        for color in ( chess.WHITE, chess.BLACK )
    }

Game.get_ai_state = get_ai_state
del get_ai_state

In [ ]:
def make_move_human(self) -> None:
    input_prompt = 'Please input your move: '
    while True:
        try:
            move = chess.Move.from_uci(input(input_prompt))
            self.board.push_legal(move)
            return
        except ValueError:
            input_prompt = 'Illegal move, please try again: '
                
Game.make_move_human = make_move_human
del make_move_human

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>